In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from statistics import mode
pd.set_option('display.max_rows', 200)

In [2]:
#downloading initial file.
df_first=pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_221008.txt',sep=',')
# second one to fill the empty rows after diff()
df_m1=pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_221001.txt',sep=',')


In [3]:
df_first.shape

(211003, 11)

In [4]:
df_m1.shape

(211381, 11)

In [67]:
#concatenating files for the initail and -1 week to get values for the first row after diff()
df=pd.concat([df_m1,df_first],axis=0,ignore_index=True)
df.reset_index(inplace=True,drop=True)

In [68]:
df.shape

(422384, 11)

In [69]:
df.isnull().sum()

C/A                                                                     0
UNIT                                                                    0
SCP                                                                     0
STATION                                                                 0
LINENAME                                                                0
DIVISION                                                                0
DATE                                                                    0
TIME                                                                    0
DESC                                                                    0
ENTRIES                                                                 0
EXITS                                                                   0
dtype: int64

No null values

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422384 entries, 0 to 422383
Data columns (total 11 columns):
 #   Column                                                                Non-Null Count   Dtype 
---  ------                                                                --------------   ----- 
 0   C/A                                                                   422384 non-null  object
 1   UNIT                                                                  422384 non-null  object
 2   SCP                                                                   422384 non-null  object
 3   STATION                                                               422384 non-null  object
 4   LINENAME                                                              422384 non-null  object
 5   DIVISION                                                              422384 non-null  object
 6   DATE                                                                  422384 non-null  objec

All columns except DATE and TIME are in the correct format. Let's unite that columns and change to datetime.

In [71]:
# snake_case column names
df.columns=df.columns.str.lower().str.replace(" ","")
# converting date to datetime format
df["datetime"]=df["date"]+" "+df["time"]
df.datetime=pd.to_datetime(df["datetime"])
df.drop(columns=["date","time"],inplace=True)




In [72]:
df.head(2)

,c/a,unit,scp,station,linename,division,desc,entries,exits,datetime
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767632,2742700,2022-09-24 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767638,2742703,2022-09-24 04:00:00


Now features are fine.

----

This code creates multindex that will help group dataframe and calculate absolute values for entries and exits instead of relevant.

In [73]:
df['stat_id']=df["station"]+" "+df["scp"]+" "+df["c/a"]
df=df.set_index(["datetime"])

df

,c/a,unit,scp,station,linename,division,desc,entries,exits,stat_id
datetime,,,,,,,,,,
2022-09-24 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767632,2742700,59 ST 02-00-00 A002
2022-09-24 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767638,2742703,59 ST 02-00-00 A002
2022-09-24 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767648,2742732,59 ST 02-00-00 A002
2022-09-24 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767680,2742803,59 ST 02-00-00 A002
2022-09-24 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767756,2742863,59 ST 02-00-00 A002
...,...,...,...,...,...,...,...,...,...,...
2022-10-07 05:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,0,804,RIT-ROOSEVELT 00-05-01 TRAM2
2022-10-07 09:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,0,804,RIT-ROOSEVELT 00-05-01 TRAM2
2022-10-07 13:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,0,804,RIT-ROOSEVELT 00-05-01 TRAM2


In [74]:
df['station'].value_counts(ascending=True).index[:25]

Index(['ORCHARD BEACH', 'BROAD CHANNEL', 'CLEVELAND ST', 'SUTTER AV',
       'INTERVALE AV', 'AVENUE X', 'ALABAMA AV', 'CHAUNCEY ST', '190 ST',
       'FOREST AVE', 'CYPRESS HILLS', 'CRESCENT ST', 'WESTCHESTER SQ',
       'NORWOOD AV', '65 ST', 'NEW LOTS', 'ATLANTIC AV', 'BUSHWICK AV',
       'MONTROSE AV', 'BRONX PARK EAST', 'ALLERTON AV', 'BURKE AV', '219 ST',
       'NEREID AV', '55 ST'],
      dtype='object')

In [75]:
orchard_beach = df[df['station'] == 'ORCHARD BEACH']

In [76]:
orchard_beach.shape

(138, 10)

In [77]:
orchard_beach['stat_id'].value_counts().index

Index(['ORCHARD BEACH 00-00-01 OB01', 'ORCHARD BEACH 00-03-00 OB01',
       'ORCHARD BEACH 00-03-01 OB01', 'ORCHARD BEACH 00-00-00 OB01',
       'ORCHARD BEACH 00-00-02 OB01'],
      dtype='object')

In [78]:
turnstyle_1 = orchard_beach[orchard_beach['stat_id'] == 'ORCHARD BEACH 00-00-01 OB01']


In [84]:
turnstyle_1

,c/a,unit,scp,station,linename,division,desc,entries,exits,stat_id
datetime,,,,,,,,,,
2022-09-24 00:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01
2022-09-24 04:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01
2022-09-24 08:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01
2022-09-24 12:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01
2022-09-24 16:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01
2022-09-24 20:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01
2022-09-25 00:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01
2022-09-25 04:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01
2022-09-25 08:00:00,OB01,R459,00-00-01,ORCHARD BEACH,6,IND,REGULAR,72864,5926,ORCHARD BEACH 00-00-01 OB01


In [87]:
turnstyle_1.iloc[0]['exits']

5926

In [85]:
turnstyle_1.iloc[-1]

c/a                                OB01
unit                               R459
scp                            00-00-01
station                   ORCHARD BEACH
linename                              6
division                            IND
desc                            REGULAR
entries                           72864
exits                              5926
stat_id     ORCHARD BEACH 00-00-01 OB01
Name: 2022-09-28 12:00:00, dtype: object

In [95]:
def total_entries_exits(df):
    turnstiles = df['stat_id'].value_counts().index
    entries_exits = []
    
    for turnstile in turnstiles:
        t_df = df[df['stat_id'] == turnstile]
        exits = t_df.iloc[-1]['exits'] - t_df.iloc[0]['exits']
        entries = t_df.iloc[-1]['entries'] - t_df.iloc[0]['entries']
        entries_exits.append([turnstile, entries, exits])
        
    return entries_exits

In [104]:
orchard_ee = pd.DataFrame(total_entries_exits(orchard_beach), columns = ['stat_id', 'total_entries', 'total_exits'])
broad_ee = pd.DataFrame(total_entries_exits(df[df['station'] == 'BROAD CHANNEL']), columns = ['stat_id', 'total_entries', 'total_exits'])
ee = pd.concat([orchard_ee, broad_ee])
ee

,stat_id,total_entries,total_exits
0,ORCHARD BEACH 00-00-01 OB01,0,0
1,ORCHARD BEACH 00-03-00 OB01,0,0
2,ORCHARD BEACH 00-03-01 OB01,0,0
3,ORCHARD BEACH 00-00-00 OB01,0,0
4,ORCHARD BEACH 00-00-02 OB01,0,0
0,BROAD CHANNEL 00-00-01 N183,978,172
1,BROAD CHANNEL 00-00-02 N183,228,51
2,BROAD CHANNEL 00-00-00 N183,107,117


In [105]:
def total_entries_exits_per_station(df):
    stations = df['station'].value_counts(ascending=True).index
    total_df = pd.DataFrame(columns =['stat_id', 'total_entries', 'total_exits', 'station'])
    
    for station in stations:
        entries_exits = total_entries_exits(df[df['station'] == station])
        new_df = pd.DataFrame(entries_exits, columns = ['stat_id', 'total_entries', 'total_exits'])
        new_df['station'] = station
        total_df = pd.concat([total_df, new_df])
        
    return total_df

In [106]:
entries_exits_df = total_entries_exits_per_station(df)

In [107]:
entries_exits_df.head()

,stat_id,total_entries,total_exits,station
0,ORCHARD BEACH 00-00-01 OB01,0,0,ORCHARD BEACH
1,ORCHARD BEACH 00-03-00 OB01,0,0,ORCHARD BEACH
2,ORCHARD BEACH 00-03-01 OB01,0,0,ORCHARD BEACH
3,ORCHARD BEACH 00-00-00 OB01,0,0,ORCHARD BEACH
4,ORCHARD BEACH 00-00-02 OB01,0,0,ORCHARD BEACH


In [108]:
entries_exits_df.shape

(5041, 4)

In [116]:
entries_exits_df.sort_values('total_entries').head(100)

,stat_id,total_entries,total_exits,station
42,125 ST 00-06-00 R258,-1863340006,-1410533131,125 ST
64,86 ST 01-06-01 C026,-261356395,-261356104,86 ST
27,LEXINGTON AV/53 00-03-00 N306,-18313811,-11671431,LEXINGTON AV/53
9,14 ST-UNION SQ 01-03-01 R221,-12445201,-18181817,14 ST-UNION SQ
8,14 ST-UNION SQ 01-03-02 R221,-7601723,-12920050,14 ST-UNION SQ
36,WALL ST 00-00-02 R111,-7211629,-10831399,WALL ST
10,167 ST 00-00-03 R283,-2159998,-1214218,167 ST
14,TIMES SQ-42 ST 01-00-03 R148,-1296906,-326171,TIMES SQ-42 ST
22,ST. GEORGE 00-00-02 S101,-1256507,-141,ST. GEORGE
21,66 ST-LINCOLN 00-00-00 R160A,-864668,-1256945,66 ST-LINCOLN


In [119]:
tt_st = df[df['station'] == 'THIRTY THIRD ST']

In [121]:
tt_st_ee = total_entries_exits(tt_st)
tt_st_ee

[['THIRTY THIRD ST 01-00-04 PTH17', -103516, -72850],
 ['THIRTY THIRD ST 01-00-00 PTH17', 0, 0],
 ['THIRTY THIRD ST 01-00-0A PTH17', 3353, 3803],
 ['THIRTY THIRD ST 01-01-05 PTH17', 15991, 2390],
 ['THIRTY THIRD ST 01-01-04 PTH17', 2720, 4542],
 ['THIRTY THIRD ST 01-01-03 PTH17', 3872, 4811],
 ['THIRTY THIRD ST 01-01-02 PTH17', 9300, 4962],
 ['THIRTY THIRD ST 01-01-01 PTH17', 17359, 5018],
 ['THIRTY THIRD ST 01-01-00 PTH17', 13104, 5769],
 ['THIRTY THIRD ST 01-00-09 PTH17', 13936, 9026],
 ['THIRTY THIRD ST 01-00-01 PTH17', 0, 0],
 ['THIRTY THIRD ST 01-00-08 PTH17', 10339, 12467],
 ['THIRTY THIRD ST 01-00-07 PTH17', 10690, 11874],
 ['THIRTY THIRD ST 01-00-06 PTH17', 13038, 12727],
 ['THIRTY THIRD ST 01-00-05 PTH17', 7711, 9082],
 ['THIRTY THIRD ST 01-00-03 PTH17', 0, 9984],
 ['THIRTY THIRD ST 01-00-02 PTH17', 0, 14634],
 ['THIRTY THIRD ST 01-01-06 PTH17', 16075, 3053]]

In [122]:
tt_st[tt_st['stat_id'] == 'THIRTY THIRD ST 01-00-04 PTH17']

,c/a,unit,scp,station,linename,division,desc,entries,exits,stat_id
datetime,,,,,,,,,,
2022-09-24 00:21:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104317,73298,THIRTY THIRD ST 01-00-04 PTH17
2022-09-24 04:33:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104334,73304,THIRTY THIRD ST 01-00-04 PTH17
2022-09-24 08:45:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104353,73323,THIRTY THIRD ST 01-00-04 PTH17
2022-09-24 12:57:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104370,73338,THIRTY THIRD ST 01-00-04 PTH17
2022-09-24 17:09:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104421,73542,THIRTY THIRD ST 01-00-04 PTH17
2022-09-24 21:21:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104533,73650,THIRTY THIRD ST 01-00-04 PTH17
2022-09-25 01:33:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104600,73650,THIRTY THIRD ST 01-00-04 PTH17
2022-09-25 05:45:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104613,73650,THIRTY THIRD ST 01-00-04 PTH17
2022-09-25 09:57:02,PTH17,R541,01-00-04,THIRTY THIRD ST,1,PTH,REGULAR,104629,73650,THIRTY THIRD ST 01-00-04 PTH17


In [123]:
uq_st = df[df['station'] == '14 ST-UNION SQ']

In [124]:
uq_st_ee = total_entries_exits(uq_st)
uq_st_ee

[['14 ST-UNION SQ 00-00-02 A035', 5598, 24316],
 ['14 ST-UNION SQ 00-00-03 A035', 9975, 17877],
 ['14 ST-UNION SQ 00-00-00 A035', 1394, 5032],
 ['14 ST-UNION SQ 00-00-01 A035', 3196, 17785],
 ['14 ST-UNION SQ 01-06-01 R221', 5879, 26924],
 ['14 ST-UNION SQ 01-06-00 R221', 3193, 47361],
 ['14 ST-UNION SQ 01-05-01 R221', 0, 116],
 ['14 ST-UNION SQ 01-05-00 R221', 1525, 0],
 ['14 ST-UNION SQ 01-03-02 R221', -7601723, -12920050],
 ['14 ST-UNION SQ 01-03-01 R221', -12445201, -18181817],
 ['14 ST-UNION SQ 01-06-02 R221', 9763, 20165],
 ['14 ST-UNION SQ 01-00-02 R221', 16908, 22531],
 ['14 ST-UNION SQ 01-00-01 R221', 17685, 18975],
 ['14 ST-UNION SQ 01-00-00 R221', 22937, 35027],
 ['14 ST-UNION SQ 05-00-03 A037', 15568, 4862],
 ['14 ST-UNION SQ 05-00-02 A037', 15320, 12047],
 ['14 ST-UNION SQ 05-00-01 A037', 14138, 19833],
 ['14 ST-UNION SQ 05-00-00 A037', 17121, 25555],
 ['14 ST-UNION SQ 01-06-04 R221', 1508, 1749],
 ['14 ST-UNION SQ 01-03-00 R221', -28046, -64182],
 ['14 ST-UNION SQ 02-00-0

In [125]:
uq_st[uq_st['stat_id'] == '14 ST-UNION SQ 01-03-00 R221']

,c/a,unit,scp,station,linename,division,desc,entries,exits,stat_id
datetime,,,,,,,,,,
2022-09-24 01:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,34429,77965,14 ST-UNION SQ 01-03-00 R221
2022-09-24 05:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,34455,78010,14 ST-UNION SQ 01-03-00 R221
2022-09-24 09:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,34513,78144,14 ST-UNION SQ 01-03-00 R221
2022-09-24 13:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,34632,78600,14 ST-UNION SQ 01-03-00 R221
2022-09-24 17:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,34829,79248,14 ST-UNION SQ 01-03-00 R221
2022-09-24 21:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,35020,79844,14 ST-UNION SQ 01-03-00 R221
2022-09-25 01:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,35144,80142,14 ST-UNION SQ 01-03-00 R221
2022-09-25 05:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,35168,80196,14 ST-UNION SQ 01-03-00 R221
2022-09-25 09:00:00,R221,R170,01-03-00,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,35202,80298,14 ST-UNION SQ 01-03-00 R221


In [127]:
uq_st[uq_st['stat_id'] == '14 ST-UNION SQ 01-03-01 R221']

,c/a,unit,scp,station,linename,division,desc,entries,exits,stat_id
datetime,,,,,,,,,,
2022-09-24 01:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12450740,18198950,14 ST-UNION SQ 01-03-01 R221
2022-09-24 05:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12450762,18199019,14 ST-UNION SQ 01-03-01 R221
2022-09-24 09:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12450821,18199231,14 ST-UNION SQ 01-03-01 R221
2022-09-24 13:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12450908,18199739,14 ST-UNION SQ 01-03-01 R221
2022-09-24 17:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12451089,18200445,14 ST-UNION SQ 01-03-01 R221
2022-09-24 21:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12451220,18201135,14 ST-UNION SQ 01-03-01 R221
2022-09-25 01:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12451319,18201482,14 ST-UNION SQ 01-03-01 R221
2022-09-25 05:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12451336,18201530,14 ST-UNION SQ 01-03-01 R221
2022-09-25 09:00:00,R221,R170,01-03-01,14 ST-UNION SQ,456LNQRW,IRT,REGULAR,12451376,18201679,14 ST-UNION SQ 01-03-01 R221


In [129]:
de_st = df[df['station'] == 'DEKALB AV']

In [130]:
de_st_ee = total_entries_exits(de_st)
de_st_ee

[['DEKALB AV 00-06-02 H023', 4898, 5165],
 ['DEKALB AV 00-06-01 H023', 12246, 702],
 ['DEKALB AV 00-06-00 H023', -9018, 5484],
 ['DEKALB AV 00-00-02 H023', 11680, 10886],
 ['DEKALB AV 00-00-01 H023', -5960, 5147],
 ['DEKALB AV 00-00-00 H023', 10432, 8477],
 ['DEKALB AV 00-06-00 C008', 15188, 18388],
 ['DEKALB AV 00-06-03 C008', 9846, 10701],
 ['DEKALB AV 00-06-02 C008', 16676, 7510],
 ['DEKALB AV 00-06-01 C008', 10028, 9667],
 ['DEKALB AV 00-00-00 C008', -3216, 20427],
 ['DEKALB AV 00-00-01 C008', 2625, 8422],
 ['DEKALB AV 00-04-00 C008', 249, 0],
 ['DEKALB AV 00-03-04 C008', 10706, 15434],
 ['DEKALB AV 00-03-03 C008', 10629, 8769],
 ['DEKALB AV 00-03-02 C008', 7285, 5172],
 ['DEKALB AV 00-03-01 C008', 4916, 3753],
 ['DEKALB AV 00-03-00 C008', 4656, 3697],
 ['DEKALB AV 00-00-03 C008', 3885, 3905],
 ['DEKALB AV 00-00-02 C008', 2562, 5386],
 ['DEKALB AV 00-04-01 C008', 0, 25]]

In [131]:
de_st[de_st['stat_id'] == 'DEKALB AV 00-06-00 H023']

,c/a,unit,scp,station,linename,division,desc,entries,exits,stat_id
datetime,,,,,,,,,,
2022-09-24 00:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926298414,856514082,DEKALB AV 00-06-00 H023
2022-09-24 04:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926298407,856514101,DEKALB AV 00-06-00 H023
2022-09-24 08:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926298340,856514141,DEKALB AV 00-06-00 H023
2022-09-24 12:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926298219,856514202,DEKALB AV 00-06-00 H023
2022-09-24 16:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926298082,856514283,DEKALB AV 00-06-00 H023
2022-09-24 20:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926297967,856514386,DEKALB AV 00-06-00 H023
2022-09-25 00:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926297922,856514438,DEKALB AV 00-06-00 H023
2022-09-25 04:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926297906,856514458,DEKALB AV 00-06-00 H023
2022-09-25 08:00:00,H023,R236,00-06-00,DEKALB AV,L,BMT,REGULAR,1926297861,856514481,DEKALB AV 00-06-00 H023


Some of the turnstile detectors were replaced/reseted. We can calculate that difference, but it will take a lot of time and since incorrect data is less than 0.5 percent of the dataset, more efficient solution is just drop it.

In [757]:
df.loc["125 ST 00-06-00 R258"].iloc[50:]


,c/a,unit,scp,station,linename,division,desc,entries,exits
datetime,,,,,,,,,
2022-10-04 09:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275772980,324331921
2022-10-04 13:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773000,324332194
2022-10-05 05:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773227,324332636
2022-10-05 09:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773365,324332903
2022-10-06 13:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773530,324333223
2022-10-06 21:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1325,254
2022-10-07 01:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1362,327
2022-10-07 05:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1375,359
2022-10-07 09:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1551,783


In [758]:
df.loc["WOODHAVEN BLVD 00-03-03 N329"].head()

,c/a,unit,scp,station,linename,division,desc,entries,exits
datetime,,,,,,,,,
2022-09-24 01:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16662066,250875
2022-09-24 05:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16662063,250887
2022-09-24 09:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16662025,250904
2022-09-24 13:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16661961,250951
2022-09-24 17:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16661870,251048


Some of the data from turnstiles is reversed in the dataset. Let's add reverse for such cases in the code below. This method gets grouped dataframe and returns absolute values for entries and exits instead of relevant.

In [759]:
res=pd.DataFrame()
def calc(temp):
    #if data if reversed - changes the order and calculates diff()
    if temp['entries'].diff().sum()<0:
        ds = pd.Series(temp['entries'].values[::-1], temp['entries'].index)       
        temp["entries_abs"]=ds.diff()
    else:
        temp['entries_abs']=temp['entries'].diff()
        
    #if data if reversed - changes the order and calculates diff()
    if temp['exits'].diff().sum()<0:
        ds = pd.Series(temp['exits'].values[::-1], temp['exits'].index)       
        temp["exits_abs"]=ds.diff()
    else:
        temp['exits_abs']=temp['exits'].diff()
    return temp

In [760]:
df=df.groupby(level=0).apply(lambda x: calc(x))


In [761]:
df

c/a  unit       scp    station  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 04:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 08:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 12:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 16:00:00  H007  R248  00-00-00       1 AV   
...                                           ...   ...       ...        ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 09:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 13:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 17:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 21:00:00  R419  R326  00-05-01  ZEREGA AV   

                                            linename division     desc  \
stat_id                 datetime                                         
1 AV 00-00-00 H007      2022-09-24 00:00:00        L      BMT  REGULAR   
                        2022-09-24 04:00:00        L      BMT  REGULAR   
                        2022-09-24 08:00:00        L      BMT  REGULAR   
                        2022-09-24 12:00:00        L      BMT  REGULAR   
                        2022-09-24 16:00:00        L      BMT  REGULAR   
...                                              ...      ...      ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        6      IRT  REGULAR   
                        2022-10-07 09:00:00        6      IRT  REGULAR   
                        2022-10-07 13:00:00        6      IRT  REGULAR   
                        2022-10-07 17:00:00        6      IRT  REGULAR   
                        2022-10-07 21:00:00        6      IRT  REGULAR   

                                              entries     exits  entries_abs  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  15841287  19006318          NaN   
                        2022-09-24 04:00:00  15841308  19006484         21.0   
                        2022-09-24 08:00:00  15841319  19006685         11.0   
                        2022-09-24 12:00:00  15841363  19007161         44.0   
                        2022-09-24 16:00:00  15841455  19007876         92.0   
...                                               ...       ...          ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        39       289          0.0   
                        2022-10-07 09:00:00        39       289          0.0   
                        2022-10-07 13:00:00        39       289          0.0   
                        2022-10-07 17:00:00        39       289          0.0   
                        2022-10-07 21:00:00        39       289          0.0   

                                             exits_abs  
stat_id                 datetime                        
1 AV 00-00-00 H007      2022-09-24 00:00:00        NaN  
                        2022-09-24 04:00:00      166.0  
                        2022-09-24 08:00:00      201.0  
                        2022-09-24 12:00:00      476.0  
                        2022-09-24 16:00:00      715.0  
...                                                ...  
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        0.0  
                        2022-10-07 09:00:00        0.0  
                        2022-10-07 13:00:00        0.0  
                        2022-10-07 17:00:00        0.0  
                        2022-10-07 21:00:00        0.0  

[422384 rows x 11 columns]

Dropping turnstiles with negative values in diff()

In [762]:
df=df.drop(df.index[df['entries_abs'] < 0])
df=df.drop(df.index[df['exits_abs'] < 0])


In [763]:
df

c/a  unit       scp    station  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 04:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 08:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 12:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 16:00:00  H007  R248  00-00-00       1 AV   
...                                           ...   ...       ...        ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 09:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 13:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 17:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 21:00:00  R419  R326  00-05-01  ZEREGA AV   

                                            linename division     desc  \
stat_id                 datetime                                         
1 AV 00-00-00 H007      2022-09-24 00:00:00        L      BMT  REGULAR   
                        2022-09-24 04:00:00        L      BMT  REGULAR   
                        2022-09-24 08:00:00        L      BMT  REGULAR   
                        2022-09-24 12:00:00        L      BMT  REGULAR   
                        2022-09-24 16:00:00        L      BMT  REGULAR   
...                                              ...      ...      ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        6      IRT  REGULAR   
                        2022-10-07 09:00:00        6      IRT  REGULAR   
                        2022-10-07 13:00:00        6      IRT  REGULAR   
                        2022-10-07 17:00:00        6      IRT  REGULAR   
                        2022-10-07 21:00:00        6      IRT  REGULAR   

                                              entries     exits  entries_abs  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  15841287  19006318          NaN   
                        2022-09-24 04:00:00  15841308  19006484         21.0   
                        2022-09-24 08:00:00  15841319  19006685         11.0   
                        2022-09-24 12:00:00  15841363  19007161         44.0   
                        2022-09-24 16:00:00  15841455  19007876         92.0   
...                                               ...       ...          ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        39       289          0.0   
                        2022-10-07 09:00:00        39       289          0.0   
                        2022-10-07 13:00:00        39       289          0.0   
                        2022-10-07 17:00:00        39       289          0.0   
                        2022-10-07 21:00:00        39       289          0.0   

                                             exits_abs  
stat_id                 datetime                        
1 AV 00-00-00 H007      2022-09-24 00:00:00        NaN  
                        2022-09-24 04:00:00      166.0  
                        2022-09-24 08:00:00      201.0  
                        2022-09-24 12:00:00      476.0  
                        2022-09-24 16:00:00      715.0  
...                                                ...  
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        0.0  
                        2022-10-07 09:00:00        0.0  
                        2022-10-07 13:00:00        0.0  
                        2022-10-07 17:00:00        0.0  
                        2022-10-07 21:00:00        0.0  

[420431 rows x 11 columns]

In [764]:
df[df["entries_abs"]<0]

,,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
stat_id,datetime,,,,,,,,,,,


In [765]:
df[df["exits_abs"]<0]

,,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
stat_id,datetime,,,,,,,,,,,


Dropping all NaN values and changing size of dataset to one week.

In [766]:
df.isnull().sum()

c/a               0
unit              0
scp               0
station           0
linename          0
division          0
desc              0
entries           0
exits             0
entries_abs    5041
exits_abs      5041
dtype: int64

In [767]:
df.dropna(inplace=True)

In [768]:
df.isnull().sum()

c/a            0
unit           0
scp            0
station        0
linename       0
division       0
desc           0
entries        0
exits          0
entries_abs    0
exits_abs      0
dtype: int64

In [769]:
df=df.reset_index(level=0).loc["2022-10-01":"2022-10-07"]

In [770]:
df

,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
datetime,,,,,,,,,,,,
2022-10-01 00:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845079,19027035,92.0,474.0
2022-10-01 04:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845098,19027234,19.0,199.0
2022-10-01 08:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845108,19027401,10.0,167.0
2022-10-01 12:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845163,19027781,55.0,380.0
2022-10-01 16:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845254,19028361,91.0,580.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-07 05:00:00,ZEREGA AV 00-05-01 R419,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,39,289,0.0,0.0
2022-10-07 09:00:00,ZEREGA AV 00-05-01 R419,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,39,289,0.0,0.0
2022-10-07 13:00:00,ZEREGA AV 00-05-01 R419,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,39,289,0.0,0.0


----

There is another problem. As you can see on the "entries" column, for the turnstile 01-05-00 at WORLD TRADE CTR Station, calculated number of entries was 2140135519, and in eight hours it changes to 8098. Probably turnstile was reseted or replased for some reasons. But that seriously affects absolute value calculation. I'll replace those values with mean for that station on the other day. The threshold to detect such fluctuations will be if that absolute value is greater than 15000. I follow the logic that it is phisically impossible to pass turnstile faster than for a second. For a period of 4 hours this gives us theoretical max amount of people at 4 * 60 * 60 = 14400.

In [771]:
df[df["station"]=='WORLD TRADE CTR'].iloc[527:530]

,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
datetime,,,,,,,,,,,,
2022-10-01 00:00:00,WORLD TRADE CTR 01-05-00 N094,N094,R029,01-05-00,WORLD TRADE CTR,ACE23,IND,REGULAR,2140135519,9,2.140127e+09,0.0
2022-10-01 08:00:00,WORLD TRADE CTR 01-05-00 N094,N094,R029,01-05-00,WORLD TRADE CTR,ACE23,IND,RECOVR AUD,8098,9,1.000000e+00,0.0
2022-10-01 12:00:00,WORLD TRADE CTR 01-05-00 N094,N094,R029,01-05-00,WORLD TRADE CTR,ACE23,IND,RECOVR AUD,8099,9,1.000000e+00,0.0


I'll drop the index to find and replace these incorrect values with mean of their station respectively.

In [773]:
df.reset_index(inplace=True)

In [774]:
for x,k in df.iterrows():
    #if absolute value of people passed through turnstile is more than 15000, replacing this value with mean.
    if df.loc[x,"entries_abs"]>15000:
        mean=df[(df["entries_abs"]<15000) & (df["station"]==k[5])]["entries_abs"].mean()
        mean=int(mean)    
        df.loc[x,"entries_abs"]=mean

114.19622641509434 10534
70.8645707376058 17000
82.92361883574341 75024
112.87530562347189 108410
115.68731848983543 155344
115.68731848983542 155350
68.19200695047785 161473
66.11011904761905 187644
87.46364379084967 196933
33.28620019436346 202791
47.330798479087456 207939


In [775]:
for x,k in df.iterrows():
    #if absolute value of people passed through turnstile is more than 15000, replacing this value with mean.
    if df.loc[x,"exits_abs"]>15000:
        mean=df[(df["exits_abs"]<15000) & (df["station"]==k[5])]["exits_abs"].mean()
        mean=int(mean)    
        df.loc[x,"exits_abs"]=mean

No incorrect data.

In [776]:
df[df["entries_abs"]>15000]

,datetime,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs


In [778]:
df[df["exits_abs"]>15000]

,datetime,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs


Changing entries_abs and exits_abs to int

In [780]:
df["entries_abs"]=df["entries_abs"].astype(int)
df["exits_abs"]=df["exits_abs"].astype(int)

In [781]:
df.head(1)

,datetime,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
0,2022-10-01,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845079,19027035,92,474


Now, let's drop: 
- entries and exits (we already used them)
- division - this feature contains information about station owner. This won't be useful for this project.
- c/a - this feature contains information about electrical station that supplies energy for current station. This won't be useful for this project either.
- desc - represent the "REGULAR" scheduled audit event. This won't be useful for this project.


In [782]:
df.drop(columns=["entries","exits","c/a","division","desc"],inplace=True)
df

,datetime,stat_id,unit,scp,station,linename,entries_abs,exits_abs
0,2022-10-01 00:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,92,474
1,2022-10-01 04:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,19,199
2,2022-10-01 08:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,10,167
3,2022-10-01 12:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,55,380
4,2022-10-01 16:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,91,580
...,...,...,...,...,...,...,...,...
210010,2022-10-07 05:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
210011,2022-10-07 09:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
210012,2022-10-07 13:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
210013,2022-10-07 17:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0


Saving file to csv.

In [785]:
df.to_csv('../data/df_cleaned.csv',index=False)

In [786]:
dd=pd.read_csv('../data/df_cleaned.csv',index_col="datetime")
dd

,stat_id,unit,scp,station,linename,entries_abs,exits_abs
datetime,,,,,,,
2022-10-01 00:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,92,474
2022-10-01 04:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,19,199
2022-10-01 08:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,10,167
2022-10-01 12:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,55,380
2022-10-01 16:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,91,580
...,...,...,...,...,...,...,...
2022-10-07 05:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
2022-10-07 09:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
2022-10-07 13:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
